In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objects as go
import json
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pyarrow.parquet as pq
nltk.download("vader_lexicon")



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [66]:
with open(r"C:\Users\MSI\Documents\Proyectos\Datasets\Steam_Data_Games\australian_user_reviews.json",'r', encoding='utf-8') as review:
    lines_review = review.readlines()
list_review = [eval(line.strip()) for line in lines_review]
data_users_review = pd.DataFrame(list_review).drop(columns=['user_url'])

In [107]:
data_users_review.info()

,user_id,reviews
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...
59300,76561198312638244,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59301,76561198312638244,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59302,LydiaMorley,"{'funny': '1 person found this review funny', ..."
59303,LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [68]:
data_users_review = data_users_review.explode('reviews')

In [69]:
data_users_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  59333 non-null  object
 1   reviews  59305 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [70]:
data_users_review.dropna(inplace=True)

In [114]:
data_users_review

,user_id,reviews
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...
59300,76561198312638244,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59301,76561198312638244,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59302,LydiaMorley,"{'funny': '1 person found this review funny', ..."
59303,LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [72]:
data_users_review.reset_index(drop=True, inplace=True)

In [73]:
claves_deseadas = ['item_id', 'review','recommend']

data_reviews = pd.DataFrame(list(map(lambda d: {clave: d[clave] for clave in claves_deseadas}, data_users_review.reviews)))
data_reviews

,item_id,review,recommend
0,1250,Simple yet with great replayability. In my opi...,True
1,22200,It's unique and worth a playthrough.,True
2,43110,Great atmosphere. The gunplay can be a bit chu...,True
3,251610,I know what you think when you see this title ...,True
4,227300,For a simple (it's actually not all that simpl...,True
...,...,...,...
59300,70,a must have classic from steam definitely wort...,True
59301,362890,this game is a perfect remake of the original ...,True
59302,273110,had so much fun plaing this and collecting res...,True
59303,730,:D,True


In [74]:
data_all_reviews = pd.merge(data_users_review['user_id'], data_reviews,left_index=True,right_index=True, how='inner')

In [75]:
def clean_text(text):
  text = re.sub(r'^RT[\s]+', '', text)
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'#', '', text)
  text = re.sub(r'@[A-Za-z0-9]+', '', text)
  return text


def sentimientos(texto):
    analizador_sentimientos = SentimentIntensityAnalyzer()
    
    if pd.isnull(texto) or texto == "":
        return 1  # Valor neutral si el texto está vacío o es NaN
    elif isinstance(texto, str):
        # Realizamos análisis de sentimiento
        sentiment = analizador_sentimientos.polarity_scores(texto)
        compound_score = sentiment["compound"]
        if compound_score > 0:
            return 2 # Positivo
        elif compound_score < -0:
            return 0 # Negativo
        else:
            return 1 # Neutral
    else:
        return 1  # Valor neutral para datos que no son de tipo cadena

In [117]:
data_all_reviews

,user_id,item_id,recommend,sentiment
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,2
...,...,...,...,...
59300,76561198312638244,70,True,2
59301,76561198312638244,362890,True,2
59302,LydiaMorley,273110,True,2
59303,LydiaMorley,730,True,2


In [77]:
data_all_reviews.review = data_all_reviews.review.apply(clean_text)
data_all_reviews['sentiment'] = data_all_reviews.review.apply(sentimientos)
data_all_reviews.drop(columns=['review'], inplace=True)


In [118]:
data_all_reviews.to_parquet('data_reviews.parquet')

In [79]:
with open(r"C:\Users\MSI\Documents\Proyectos\Datasets\Steam_Data_Games\australian_users_items.json", 'r', encoding= 'utf-8') as f:
    lines_items = f.readlines()
    
list_items = [eval(line.strip()) for line in lines_items]
data_users_items = pd.DataFrame(list_items).drop(columns=['user_url','steam_id','items_count'])

In [81]:
desanidado = []
for index, row in data_users_items.iterrows():
    user_id = row['user_id']
    for df_items_dict in row['items']:
        df_items_dict.pop('playtime_2weeks')
        df_items_dict['user_id'] = user_id
        desanidado.append(df_items_dict)

data_items = pd.DataFrame(desanidado)

In [120]:
data_items

In [82]:
with open(r"C:\Users\MSI\Documents\Proyectos\Datasets\Steam_Data_Games\output_steam_games.json", 'r', encoding='utf-8') as games:
    data = games.readlines()
datos = [json.loads(line.strip()) for line in data]
steam_games =pd.DataFrame(datos)

In [83]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [84]:
steam_games.drop(columns=['url','reviews_url','publisher','early_access'], inplace=True)

In [85]:
steam_games.dropna(how='all', inplace=True)

In [86]:
steam_games.reset_index(drop=True, inplace=True)

In [87]:
steam_games.price.value_counts()

price
4.99           4278
9.99           3902
2.99           3429
0.99           2607
1.99           2541
               ... 
74.76             1
32.99             1
14.95             1
26.99             1
Free to Use       1
Name: count, Length: 162, dtype: int64

In [88]:
def replace_prices(row):

    try:
        return float(row)
    except:
        price_str = row.lower().strip()
        if ('free' in price_str) or ('demo' in  price_str):
            return 0
        else:
            return None
    

In [89]:
steam_games.price = steam_games.price.apply(replace_prices)

In [90]:
steam_games = steam_games.loc[(steam_games[['title','id']].drop_duplicates().dropna().index),:]

In [91]:
def release_modif(row):
    try:
        return pd.to_datetime(row,format='%Y-%m-%d')
    except:
        return None

In [92]:
steam_games.release_date = steam_games.release_date.map(release_modif)

In [93]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30083 entries, 0 to 32133
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28849 non-null  object        
 1   app_name      30083 non-null  object        
 2   title         30083 non-null  object        
 3   release_date  29780 non-null  datetime64[ns]
 4   tags          29922 non-null  object        
 5   specs         29414 non-null  object        
 6   price         28815 non-null  float64       
 7   id            30083 non-null  object        
 8   developer     28834 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 2.3+ MB


In [121]:
steam_games

,genres,app_name,title,release_date,tags,specs,price,id,developer
0,"[action, casual, indie, simulation, strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[strategy, action, indie, casual, simulation]",[single-player],4.99,761140,Kotoshiro
1,"[free to play, indie, rpg, strategy]",Ironbound,Ironbound,2018-01-04,"[free to play, strategy, indie, rpg, card game...","[single-player, multi-player, online multi-pla...",0.00,643980,Secret Level SRL
2,"[casual, free to play, indie, simulation, sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[free to play, simulation, sports, casual, ind...","[single-player, multi-player, online multi-pla...",0.00,670290,Poolians.com
3,"[action, adventure, casual]",弹炸人2222,弹炸人2222,2017-12-07,"[action, adventure, casual]",[single-player],0.99,767400,彼岸领域
5,"[action, adventure, simulation]",Battle Royale Trainer,Battle Royale Trainer,2018-01-04,"[action, adventure, simulation, fps, shooter, ...","[single-player, steam achievements]",3.99,772540,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...,...
32129,"[action, adventure, casual, indie]",Kebab it Up!,Kebab it Up!,2018-01-04,"[action, indie, casual, violent, adventure]","[single-player, steam achievements, steam cloud]",1.99,745400,Bidoniera Games
32130,"[casual, indie, simulation, strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[strategy, indie, casual, simulation]","[single-player, steam achievements]",1.99,773640,"Nikita ""Ghost_RUS"""
32131,"[casual, indie, strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[strategy, indie, casual]","[single-player, steam achievements, steam clou...",4.99,733530,Sacada
32132,"[indie, racing, simulation]",Russian Roads,Russian Roads,2018-01-04,"[indie, simulation, racing]","[single-player, steam achievements, steam trad...",1.99,610660,Laush Dmitriy Sergeevich


In [95]:
def filter_lists(column_value):
    if isinstance(column_value, list):
        
        column_value_low = []
        for indx in column_value:
            if indx in column_value_low:
                continue
            else:
                column_value_low.append(indx.lower())
        return column_value_low
    else:
        return []

In [96]:
steam_games['genres'] = steam_games['genres'].apply(filter_lists).dropna()
steam_games['tags'] = steam_games['tags'].apply(filter_lists).dropna()
steam_games['specs'] = steam_games['specs'].apply(filter_lists).dropna()



In [123]:
steam_games

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

In [98]:
def porcentaje_juegos_gratis(precios):
    juegos_gratis = precios[precios == 0].count()
    juegos_totales = len(precios)
    porcentaje = (juegos_gratis / juegos_totales) * 100 if juegos_totales > 0 else 0
    return f'{porcentaje:.1f}%'


def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [99]:
def userdata(user_id):
    
    user_opins = pd.merge(data_all_reviews.drop(columns='sentiment'),steam_games.drop(columns=['genres','app_name','title','release_date','tags','specs','developer']), how='left', left_on='item_id',right_on='id').query('user_id == @user_id')

    user_paid = user_opins.price.sum()
    user_recom = (len(user_opins[user_opins.recommend==True]) / len(user_opins))*100
    user_items = len(user_opins)

    return {"Usuario": user_id,"Dinero Gastado": user_paid,"% de recomendación: ": user_recom, "Cantidad de Items":user_items}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [100]:
def UserForGenre(genre):
    genre = genre.lower()
    User_gen = pd.merge(data_items, steam_games[steam_games.genres.map(lambda genr: genr == [genre])].loc[:,('id','release_date')], left_on='item_id', right_on='id')

    user = User_gen[['user_id','playtime_forever']].groupby('user_id')['playtime_forever'].sum().sort_values(ascending=False).head(1).index[0]
    registro = User_gen.query('user_id==@user')[['release_date','playtime_forever']].groupby(User_gen.query('user_id==@user')['release_date'].dt.year)['playtime_forever'].sum()

    return user,registro

def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [101]:
def best_developer_year(year):
    dev_gen = pd.merge(data_all_reviews, steam_games.loc[:,('id','release_date','developer')], left_on='item_id', right_on='id')[['recommend','sentiment','release_date','developer']].query('release_date.dt.year == @year and recommend==True and recommend==True and sentiment==2')['developer'].value_counts().head(3).to_dict()

    return dev_gen


def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [102]:
def developer_reviews_analysis(developer):
    developer = developer.lower()
    dev_dc = pd.merge(data_all_reviews, steam_games.loc[:,('id','release_date','developer')], left_on='item_id', right_on='id')[['sentiment','developer']].query('developer == @developer').groupby('developer').value_counts().to_dict()

    dev_neg = dev_dc.get((developer,0))
    dev_pos = dev_dc.get((developer,2))

    return {developer:[dev_neg,dev_pos]}
